# Analysis_Dataset_Stats <a class="tocSkip">

## Imports \& Config

In [ ]:
# Set editor width to something sane
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
import sys
# get root repository path
a = !pwd
repo_root = a[0].rsplit('ProGraML', maxsplit=1,)[0] + 'ProGraML'
print(repo_root)
#insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, repo_root)
repo_root = Path(repo_root)


In [ ]:
import pickle

import numpy as np
from matplotlib import pyplot as plt
import tqdm
import torch
from torch_geometric.data import Data, DataLoader

In [ ]:
from deeplearning.ml4pl.models.ggnn.ggnn_config_poj104 import GGNNConfig
from deeplearning.ml4pl.models.ggnn.run_ggnn_poj104 import Learner
from deeplearning.ml4pl.models.ggnn.ggnn_modules import GGNNModel
from deeplearning.ml4pl.poj104.dataset import POJ104Dataset


## Analyze Dataset Statistics

### Notable tokens and their statements

```
unreachable,232
ret void,263
!UNK,8564
!IDENTIFIER, 8565
magic/root/ I forgot, 8567
```

In [ ]:
def print_dataset_stats(dataset):
    print(dataset)
    x = []
    y = []
    for i, data in enumerate(dataset):
        x.append(data.x.squeeze(dim=1))
        y.append(data.y.item())
    x = torch.cat(x)
    
    y = torch.tensor(y)
    ybins = torch.bincount(y, minlength=104)
    print(ybins)
    
    #print(x.size())
    bins = torch.bincount(x[:, 0], minlength=8568)

    #print('token, count, percentage of whole')
    s = torch.sum(bins)
    unique = 0
    for i, c in enumerate(bins):
        if c:
            unique += 1
            #print(f"{i}  {c.item()}   {(c*100/s.float()).item():.3f}")
    print(f"No of unique tokens {unique}/8568.")
    print(f"No of tokens in ds: {s:,}.")
    print(f"No of !UNK in data: {bins[8564].item():,} -- [{bins[8564].item()*100/s.item():.2f}%].")
    print(f"No of !ID in data: {bins[8565].item():,} -- [{bins[8565].item()*100/s.item():.2f}%].")

    plt.figure(figsize=(16, 4))
    plt.bar(range(105), ybins)
    plt.show()

    plt.figure(figsize=(32, 4))
    plt.plot(bins) #, ls='')
    plt.grid(which='both', axis='y')
    plt.yscale('log')
    plt.yticks()
    #plt.plot(bad_log, ls='dotted')
    plt.show()

### Validation Set

In [ ]:
ds_dir = repo_root / 'deeplearning/ml4pl/poj104/classifyapp_data'
valid_data = POJ104Dataset(ds_dir, 'val')


In [ ]:
print_dataset_stats(valid_data)

### Test Set

In [ ]:
test_data = POJ104Dataset(ds_dir, 'test')

In [ ]:
print_dataset_stats(test_data)

### Train Set

In [ ]:
train_data = POJ104Dataset(ds_dir, 'train')

In [ ]:
print_dataset_stats(train_data)

## TBD

In [ ]:
jan20_test_data = POJ104Dataset(root=repo_root / 'deeplearning/ml4pl/poj104/classifyapp_data/processed_jan20_9_inconsistent', split='test')
jan9_test_data = POJ104Dataset(root=repo_root / 'deeplearning/ml4pl/poj104/classifyapp_data', split='test')

In [ ]:
jan20_test_data[0]

In [ ]:
import networkx.algorithms.isomorphism as iso

from torch_geometric.utils import convert




for i, (a, b) in enumerate(zip(jan9_test_data, jan20_test_data)):
    #an = convert.to_networkx(a)
    #bn = convert.to_networkx(b)
    #print(i, iso.is_isomorphic(an, bn))
    
    assert torch.all(a.edge_index[0,:] == b.edge_index[0, :]), f"{i}\n{a.edge_index[0,:] == b.edge_index[0, :]}"
    
    #d9x = np.sort(d9.x[:,0].numpy())
    #d20x = np.sort(d20.x[:,0].numpy())
    #mask = d9x == d20x
    #if not np.all(mask):
    #    print(d9x[np.invert(mask)])
    #    print(d20x[np.invert(mask)])
#    print((data9data20))